In [10]:
!pip install gradio

import gradio as gr

# Step 1: Function to process form responses
def analyze_allergy_form(season, coat, breed, symptoms):
    risk_score = 0

    # Scoring for season
    if season in ["Spring", "Summer"]:
        risk_score += 2
    elif season == "Monsoon":
        risk_score += 1

    # Scoring for coat
    if coat in ["Long coat", "Allergy-prone"]:
        risk_score += 2

    # Scoring for specific allergy-prone breeds
    allergy_prone_breeds = ["German Shepherd", "Labrador", "Pug", "Beagle"]
    if breed in allergy_prone_breeds:
        risk_score += 1

    # Scoring for symptoms
    if symptoms:
        risk_score += len(symptoms)

    # Determine risk level
    if risk_score <= 2:
        risk_level = "Low"
    elif 3 <= risk_score <= 4:
        risk_level = "Medium"
    else:
        risk_level = "High"

    summary = f"Season: {season}\nCoat: {coat}\nBreed: {breed}\nSymptoms: {', '.join(symptoms) if symptoms else 'None'}\nPredicted Allergy Risk: {risk_level}"

    return {"season": season, "coat": coat, "breed": breed, "symptoms": symptoms, "risk": risk_level}, summary


# Step 2: Chatbot logic
def allergy_chatbot(history, user_input, form_data):
    history = history or []
    if not form_data:
        response = "Please fill out the Allergy Risk Form first."
    else:
        response = ""
        # Keyword-based tips
        if "itch" in user_input.lower():
            response = "🐾 Itching could be due to pollen or dust allergies. Try wiping paws and fur after walks."
        elif "sneeze" in user_input.lower():
            response = "🤧 Sneezing may indicate pollen allergies. Consider indoor play on high pollen days."
        elif "lethargy" in user_input.lower():
            response = "😴 Lethargy can be a sign of seasonal allergy fatigue. Monitor water intake and appetite."
        else:
            response = "Thanks for sharing! I'll keep that in mind based on the allergy risk."

        # Vet recommendation based on risk
        if form_data["risk"] == "High":
            response += "\n⚠️ Allergy risk is HIGH. Please take your dog to a veterinarian."
        elif form_data["risk"] == "Medium":
            response += "\n🔍 Allergy risk is MEDIUM. Monitor closely and consult a vet if symptoms worsen."
        else:
            response += "\n✅ Allergy risk is LOW. Maintain regular care."

        # Add form summary
        symptoms_str = ", ".join(form_data["symptoms"]) if form_data["symptoms"] else "None"
        response += f"\n\n📋 [Form Summary]\nSeason: {form_data['season']}\nCoat: {form_data['coat']}\nBreed: {form_data['breed']}\nSymptoms: {symptoms_str}\nRisk: {form_data['risk']}"

    history.append((user_input, response))
    return history, history


# Step 3: Build Gradio UI (keeping same layout)
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("<h1 style='text-align:center'>🐶 Pet Seasonal Allergy Predictor</h1>")

    with gr.Tab("Allergy Risk Form"):
        season = gr.Dropdown(["Summer", "Monsoon", "Winter", "Spring"], label="Current Season")
        coat = gr.Dropdown(["Short coat", "Long coat", "Allergy-prone"], label="Dog's Coat")
        breed = gr.Dropdown(["German Shepherd", "Labrador", "Pug", "Golden Retriever", "Beagle", "Indie", "Other"], label="Dog Breed")
        symptoms = gr.CheckboxGroup(["Itching", "Sneezing", "Red eyes", "Lethargy", "Loss of appetite"], label="Observed Symptoms")

        form_output_data = gr.State()
        form_output_text = gr.Textbox(label="Allergy Risk Prediction")
        form_btn = gr.Button("Analyze Risk")
        form_btn.click(analyze_allergy_form, inputs=[season, coat, breed, symptoms], outputs=[form_output_data, form_output_text])

    with gr.Tab("Pet Behaviour Chatbot"):
        chatbot = gr.Chatbot()
        state = gr.State()
        chat_input = gr.Textbox(label="Describe your pet's behaviour")
        send_btn = gr.Button("Send")
        send_btn.click(allergy_chatbot, inputs=[state, chat_input, form_output_data], outputs=[chatbot, state])

demo.launch(share=True)


/tmp/ipython-input-726074343.py:90: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a90977397d66bd7779.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
